In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix 
from bs4 import BeautifulSoup # BeautifulSoup is in bs4 package 
import requests
from urllib.request import Request, urlopen
from time import sleep
from selectorlib import Extractor
import json
from fake_useragent import UserAgent

In [9]:
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:50]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies
px=get_proxies()
print(px)

{'95.31.5.29:54651', '103.141.108.161:3127', '188.168.56.82:55443', '62.171.173.188:8866', '119.82.252.122:36182', '117.6.161.118:53281', '91.142.174.224:41576', '51.75.147.33:3128', '118.97.100.83:35220', '159.224.243.185:37793', '1.10.186.35:37235', '178.128.125.16:34027', '46.21.153.16:3128', '182.52.51.10:61124', '103.208.200.114:23500', '179.184.230.194:46365', '81.163.62.136:41258', '103.57.70.231:39143', '103.241.227.117:6666', '193.56.255.181:3128', '36.37.74.60:8080', '54.67.62.76:20017', '187.94.16.59:39665', '85.173.165.36:46330', '36.94.35.217:55418', '90.189.110.170:3128', '103.240.77.98:30093', '52.156.97.118:1080', '197.248.184.157:53281', '202.75.97.82:47009', '200.73.129.128:8080', '103.250.158.21:6666', '103.87.48.45:35844', '195.60.174.123:39635', '1.10.188.52:32163', '197.157.219.169:48625', '103.220.204.101:59570', '175.100.72.95:57938', '46.29.11.254:57849', '185.17.134.149:45984', '38.132.99.207:3128', '101.255.103.201:53281', '81.163.62.221:34011', '95.9.194.13:

In [10]:
from itertools import cycle
import traceback
#If you are copy pasting proxy ips, put in the list below
#proxies = ['121.129.127.209:80', '124.41.215.238:45169', '185.93.3.123:8080', '194.182.64.67:3128', '106.0.38.174:8080', '163.172.175.210:3128', '13.92.196.150:8080']
proxies = get_proxies()
proxy_pool = cycle(proxies)

url = 'https://httpbin.org/ip'
for i in range(1,11):
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    print("Request #%d"%i)
    goodproxies=[]
    try:
        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        #print(response.json())
        print(response.json()['origin'])
        #goodproxies.append(response.json['origin'])
    except:
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error")
goodproxies

Request #1
95.31.5.29
Request #2
103.141.108.58
Request #3
188.168.56.82
Request #4
Skipping. Connnection error
Request #5
119.82.252.122
Request #6
Skipping. Connnection error
Request #7
91.142.174.224
Request #8
Skipping. Connnection error
Request #9
Skipping. Connnection error
Request #10
Skipping. Connnection error


[]

In [61]:
goodproxies=['116.12.236.210','181.129.74.58','117.121.202.44','81.163.25.22','163.172.93.129']

In [136]:
txt='https://www.amazon.com/s?k=Processor&ref=nb_sb_noss_2 \n'
for j in range(300):
    txt+=('https://www.amazon.com/s?k=Processor&i=computers-intl-ship&page='+str(j+2)+'&qid=1594555641&ref=sr_pg_'+str(j+2))
    if j!=299:
        txt+='\n'
text_file = open("computer_urls.txt", "w")
text_file.write(txt)
text_file.close()
#print(txt)


In [137]:
e = Extractor.from_yaml_file('search_results.yml')

def scrape(url):  
    ua=UserAgent()
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
   # proxies = {
   # "http": goodproxies[0],
   # "https": goodproxies[0],}
    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    #if r.status_code > 500:
    if "To discuss automated access to Amazon data please contact" in r.text:
        #print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
       # else:
        print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open("computer_urls.txt",'r') as urllist, open('search_results_output_1.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=Processor&i=computers-intl-ship&page=151&qid=1594555641&ref=sr_pg_151 must have been blocked by Amazon as the status code was 200
Page https://www.amazon.com/s?k=Processor&i=computers-intl-ship&page=158&qid=1594555641&ref=sr_pg_158 must have been blocked by Amazon as the status code was 200


In [2]:
processors = pd.read_json('search_results_output_1.jsonl', orient='records')

#view DataFrame
processors.loc[processors['rating'].notnull(), 'rating'] = processors['rating'].str[0:3].astype(float)
processors.loc[processors['price'].notnull(), 'price'] = processors['price'].str[1:]
processors.loc[processors['price'].notnull(), 'price'] = processors['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(processors)):
    if 'picassoRedirect' in processors['url'][j]:
        ASIN.append(((processors['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((processors['url'][j]).split("/")[3])
processors['ASIN']=ASIN
processors.loc[processors['reviews'].notnull(), 'reviews'] = processors['reviews'].str.replace(',','')
proc_cut = processors.dropna(how='any',axis=0)
#view DataFrame
proc_cut.loc[proc_cut['reviews'].notnull(), 'reviews'] = proc_cut['reviews'].astype(int)
proc_cut.info()
processors.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3861 entries, 0 to 4998
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       3861 non-null   object
 1   url         3861 non-null   object
 2   rating      3861 non-null   object
 3   reviews     3861 non-null   object
 4   price       3861 non-null   object
 5   search_url  3861 non-null   object
 6   ASIN        3861 non-null   object
dtypes: object(7)
memory usage: 241.3+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,"Tablet 9 Inch,Android 10.0 Tablets,9"" IPS Full...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,5.0,1,79.99,https://www.amazon.com/s?k=Processor&ref=nb_sb...,B08T5Q282M
1,"AMD Ryzen 5 3600 6-Core, 12-Thread Unlocked De...",/AMD-Ryzen-3600-12-Thread-Processor/dp/B07STGG...,4.9,26468,None,https://www.amazon.com/s?k=Processor&ref=nb_sb...,B07STGGQ18
2,Intel Core i9 i9-10980XE Octadeca-core (18 Cor...,/Intel-Core-i9-10980XE-Octadeca-core-Processor...,None,None,1034.99,https://www.amazon.com/s?k=Processor&ref=nb_sb...,B08DG2HRJN
3,"AMD Ryzen 7 3700X 8-Core, 16-Thread Unlocked D...",/AMD-Ryzen-3700X-16-Thread-Processor/dp/B07SXM...,4.9,17448,None,https://www.amazon.com/s?k=Processor&ref=nb_sb...,B07SXMZLPK
4,AMD Ryzen 5 2600 Processor with Wraith Stealth...,/AMD-Processor-Wraith-Stealth-Cooler/dp/B07B41...,4.8,18729,None,https://www.amazon.com/s?k=Processor&ref=nb_sb...,B07B41WS48


In [89]:
proc_cut=proc_cut.drop_duplicates(subset='title',inplace=False)
proc_cut[ (proc_cut['price']>100) & proc_cut['title'].str.contains('Ryzen') ].sort_values(by='reviews', ascending=False)['title'].head(50)

35      AMD Ryzen 5 3600X 6-Core, 12-Thread Unlocked D...
36      AMD Ryzen 5 2600X Processor with Wraith Spire ...
135     Lenovo Flex 5 14" 2-in-1 Laptop, 14.0" FHD (19...
147     Lenovo IdeaPad 3 14" Laptop, 14.0" FHD 1920 x ...
37      AMD Ryzen 5 3600XT 6-core, 12-threads unlocked...
2391    Cooler Master MasterLiquid ML360P Silver Editi...
49      AMD Ryzen 3 3100 4-Core, 8-Thread Unlocked Des...
1130    Acer Swift 3 Thin & Light Laptop, 14" Full HD ...
1136    MSI Performance Gaming AMD Ryzen 1st, 2nd, and...
8       AMD Ryzen 9 3900XT 12-core, 24-Threads Unlocke...
304           AMD Ryzen 5 1600X Processor (YD160XBCAEWOF)
186     MSI MAG B550 TOMAHAWK Gaming Motherboard (AMD ...
60      AMD Ryzen 5 5600X 6-core, 12-Thread Unlocked D...
1135    iBUYPOWER Gaming PC Computer Desktop Trace 4 9...
1657    Skytech Archangel Gaming Computer PC Desktop –...
46      AMD Ryzen 7 3800XT 8-core, 16-Threads Unlocked...
44      AMD Ryzen 7 5800X 8-core, 16-Thread Unlocked D...
2345    SkyTec

In [115]:
proc_cut['norm_review']=(proc_cut['rating']*proc_cut['reviews']+6)/(proc_cut['reviews']+2)
proc_cut[ (proc_cut['price']>10) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

2326    Noctua NF-A12x25 PWM, Premium Quiet Fan, 4-Pin...
1654    Noctua NH-U12S chromax.Black, 120mm Single-Tow...
1198    Samsung 32GB DDR4 2666MHz RAM Memory Module fo...
1792    Noctua NH-U12A, Premium CPU Cooler with High-P...
3074    Noctua NF-A12x15 PWM, Premium Quiet Slim Fan, ...
2212    Noctua NH-D15S, Premium Dual-Tower CPU Cooler ...
2739    Noctua NH-U9S, Premium CPU Cooler with NF-A9 9...
2461    Noctua NH-U12S SE-AM4, Premium CPU Cooler for ...
4111    der8auer Delid-Die-Mate 2 Integrated Heatsprea...
1772    Noctua NH-U9S chromax.Black, 92mm Single-Tower...
2285    Kingston KSM26ED8/16ME Server Premier - DDR4-1...
254     Intel Core i5-10500 Desktop Processor 6 Cores ...
3616    anidees Aluminum Heat Sinks fanless Pi Case wi...
2220    Crucial MX500 250GB 3D NAND SATA 2.5 Inch Inte...
1871    Crucial 32GB Kit (16GBx2) DDR4 3200 MT/s (PC4-...
1720    Gigastone 512GB Micro SD Card, Gaming Plus, Ni...
1125    Raspberry Pi 4 Model B 2019 Quad Core 64 Bit W...
3408    Crucia

In [164]:
txt=''
for j in range(len(processors)):
    txt+=('https://www.amazon.com/dp/'+processors['ASIN'].iloc[j]+'\n')
txt=txt[:-1]
text_file = open("full_computer_urls.txt", "w")
text_file.write(txt)
text_file.close()
#print(txt)

In [195]:
e = Extractor.from_yaml_file('processor_results.yml')

def scrape(url):  
    ua=UserAgent()
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
   # proxies = {
   # "http": goodproxies[0],
   # "https": goodproxies[0],}
    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    #if r.status_code > 500:
    if "To discuss automated access to Amazon data please contact" in r.text:
        #print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
       # else:
        print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open("full_computer_urls.txt",'r') as urllist, open('computer_results_output.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.readlines(3000):
        data = scrape(url) 
        if data:
            json.dump(data,outfile)
            outfile.write(", \n")
                #sleep(2)
    outfile.write('{} ]')

In [3]:
allcomputers = pd.read_json('computer_results_output.jsonl', orient='records')
allcomputers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        82 non-null     object 
 1   price                       55 non-null     object 
 2   short_description           82 non-null     object 
 3   images                      82 non-null     object 
 4   rating                      80 non-null     object 
 5   number_of_reviews           75 non-null     object 
 6   availability                77 non-null     object 
 7   frequently-bought-together  0 non-null      float64
 8   bought-together-link        0 non-null      float64
 9   variants                    60 non-null     object 
 10  product_description         73 non-null     object 
 11  sales_rank                  79 non-null     object 
 12  link_to_all_reviews         74 non-null     object 
dtypes: float64(2), object(11)
memory usag

In [139]:
txt='https://www.amazon.com/s?k=laptops&i=computers-intl-ship&ref=nb_sb_noss_1 \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=laptops&i=computers-intl-ship&page='+str(j+2)+'&qid=1613492391&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Laptops.txt", "w")
text_file.write(txt)
text_file.close()

In [140]:
e = Extractor.from_yaml_file('search_results.yml')

def scrape(url):  
    ua=UserAgent()
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
   # proxies = {
   # "http": goodproxies[0],
   # "https": goodproxies[0],}
    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    #if r.status_code > 500:
    if "To discuss automated access to Amazon data please contact" in r.text:
        #print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
       # else:
        print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open('Laptops.txt','r') as urllist, open('search_results_output_2.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

In [4]:
laptops = pd.read_json('search_results_output_2.jsonl', orient='records')
#view DataFrame
laptops.loc[laptops['rating'].notnull(), 'rating'] = laptops['rating'].str[0:3].astype(float)
laptops.loc[laptops['price'].notnull(), 'price'] = laptops['price'].str[1:]
laptops.loc[laptops['price'].notnull(), 'price'] = laptops['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(laptops)):
    if 'picassoRedirect' in laptops['url'][j]:
        ASIN.append(((laptops['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((laptops['url'][j]).split("/")[3])
laptops['ASIN']=ASIN
laptops.loc[laptops['reviews'].notnull(), 'reviews'] = laptops['reviews'].str.replace(',','')
lapt_cuz = laptops.dropna(how='any',axis=0)
#view DataFrame
lapt_cuz.loc[lapt_cuz['reviews'].notnull(), 'reviews'] = lapt_cuz['reviews'].astype(int)
lapt_cuz.info()
laptops.head()
#view DataFrame

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6405 entries, 0 to 7187
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       6405 non-null   object
 1   url         6405 non-null   object
 2   rating      6405 non-null   object
 3   reviews     6405 non-null   object
 4   price       6405 non-null   object
 5   search_url  6405 non-null   object
 6   ASIN        6405 non-null   object
dtypes: object(7)
memory usage: 400.3+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,CHUWI Herobook Pro 14.1 inch Windows 10 Intel ...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.0,404,349.00,https://www.amazon.com/s?k=laptops&i=computers...,B085XRL3P1
1,Used Like New Ultrabook Latitude E7240 Busines...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,5.0,1,303.00,https://www.amazon.com/s?k=laptops&i=computers...,B08QJB6MCQ
2,"ASUS VivoBook 15 F515 Thin and Light Laptop, 1...",/ASUS-i3-1005G1-Processor-Fingerprint-F515JA-A...,4.5,2908,349.99,https://www.amazon.com/s?k=laptops&i=computers...,B08H4YTTLP
3,"HP Stream 14-inch Laptop, Intel Celeron N4000,...",/HP-14-cb186nr-Brilliant-9MV74UA-ABA/dp/B084SM...,4.5,2424,319.00,https://www.amazon.com/s?k=laptops&i=computers...,B084SMMH96
4,"Lenovo Flex 5 14"" 2-in-1 Laptop, 14.0"" FHD (19...",/Lenovo-Processor-Graphics-Included-81X20005US...,4.5,3479,716.00,https://www.amazon.com/s?k=laptops&i=computers...,B086226DDB


In [61]:
lapt_cuz=lapt_cuz.drop_duplicates(subset='title',inplace=False)
lapt_cuz[ (lapt_cuz['price']>200) ].sort_values(by='reviews', ascending=False).head(20)

,title,url,rating,reviews,price,search_url,ASIN
25,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",/Acer-Display-Graphics-Keyboard-A515-43-R19L/d...,4.6,25544,399.00,https://www.amazon.com/s?k=laptops&i=computers...,B07RF1XD36
68,Lenovo Chromebook C330 2-in-1 Convertible Lapt...,/Lenovo-Chromebook-Convertible-11-6-Inch-81HY0...,4.5,6354,319.00,https://www.amazon.com/s?k=laptops&i=computers...,B07GM2J11Q
2376,Seagate FireCuda 120 SSD 2TB Internal Solid St...,/Seagate-FireCuda-Internal-Solid-State/dp/B088...,4.6,5466,360.00,https://www.amazon.com/s?k=laptops&i=computers...,B088CTFBW8
1293,Acer Flagship CB3-532 15.6inch HD Premium Chro...,/Acer-Flagship-CB3-532-Premium-Chromebook/dp/B...,4.3,5458,254.00,https://www.amazon.com/s?k=laptops&i=computers...,B06XD3LXXK
5771,"ASUS Chromebook C202 Laptop- 11.6"" Ruggedized ...",/ASUS-Chromebook-C202SA-YS02-Ruggedized-Resist...,4.3,5385,449.00,https://www.amazon.com/s?k=laptops&i=computers...,B01DBGVB7K
31,"Acer Chromebook Spin 311 Convertible Laptop, I...",/Acer-Chromebook-Convertible-Bluetooth-CP311-2...,4.7,4953,299.98,https://www.amazon.com/s?k=laptops&i=computers...,B086MBQKH2
94,HP Chromebook 14-inch Laptop with 180-Degree S...,/HP-Chromebook-180-degree-14-db0020nr-Chalkboa...,4.4,3596,246.80,https://www.amazon.com/s?k=laptops&i=computers...,B07M8QVNKG
4,"Lenovo Flex 5 14"" 2-in-1 Laptop, 14.0"" FHD (19...",/Lenovo-Processor-Graphics-Included-81X20005US...,4.5,3479,716.00,https://www.amazon.com/s?k=laptops&i=computers...,B086226DDB
616,"HP Stream 14-Inch Touchscreen Laptop, AMD Dual...",/HP-Touchscreen-Dual-Core-Processor-14-ds0090n...,4.3,3282,364.99,https://www.amazon.com/s?k=laptops&i=computers...,B07RXWCN1Q
64,"Newest HP Stream 14inch HD(1366x768) Display, ...",/HP-Stream-Dual-Core-Processor-Bluetooth/dp/B0...,4.3,3238,299.00,https://www.amazon.com/s?k=laptops&i=computers...,B07WZHW2J3


In [114]:
lapt_cuz['norm_review']=(lapt_cuz['rating']*lapt_cuz['reviews']+6)/(lapt_cuz['reviews']+2)
lapt_cuz[ (lapt_cuz['price']>100) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

937     Portable Monitor - FANGOR 15.6" Full HD 1080P ...
457     Samsung 32GB DDR4 2666MHz RAM Memory Module fo...
3476    Texbo Men's Solid Thick Full Grain Leather 17....
2064    TIDING Convertible Leather Briefcase Backpack ...
1992    G.SKILL 32GB (2 x 16G) Ripjaws Series DDR4 PC4...
601     TEAMGROUP Elite DDR4 32GB Single 2666MHz PC4-2...
4032    PNY 32GB DDR4 2666MHz Notebook Memory RAM – (M...
3277    USB-C Portable Monitor - 15.6 Inch FHD HDR Fre...
2388    Kenneth Cole Reaction Manhattan Colombian Leat...
4294    Portable Monitor, GICHAT Mini 15.6 Inch Comput...
1006    Lenovo Legion Y540 15.6" Gaming Laptop 144Hz i...
4411    Convertible Backpack Purse with Vegan Leather,...
1242    Magichold Height Adjustable Laptop Tablet Stan...
4606    Clark & Mayfield Stafford Pro Leather Laptop T...
2612          Lenovo ThinkPad USB-C Dock Gen 2 (40AS0090)
4466    Mushkin Essentials – DDR4 Laptop DRAM – 32GB (...
1652    Genuine Sam Original 32GB DDR4 3200MHz PC4-256...
6936    Fjallr

In [143]:
txt='https://www.amazon.com/s?k=monitor&i=computers-intl-ship&dc&ref=a9_sc_1 \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=monitor&i=computers-intl-ship&dc&page='+str(j+2)+'&qid=1613493148&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Monitors.txt", "w")
text_file.write(txt)
text_file.close()

In [144]:
e = Extractor.from_yaml_file('search_results.yml')

def scrape(url):  
    ua=UserAgent()
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
   # proxies = {
   # "http": goodproxies[0],
   # "https": goodproxies[0],}
    # Download the page using requests
    #print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    #if r.status_code > 500:
    if "To discuss automated access to Amazon data please contact" in r.text:
        #print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
       # else:
        print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    return e.extract(r.text)

# product_data = []
with open('Monitors.txt','r') as urllist, open('search_results_output_3.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=monitor&i=computers-intl-ship&dc&page=355&qid=1613493148&ref=sr_pg_355 must have been blocked by Amazon as the status code was 200


In [5]:
monitors = pd.read_json('search_results_output_3.jsonl', orient='records')
monitors.loc[monitors['rating'].notnull(), 'rating'] = monitors['rating'].str[0:3].astype(float)
monitors.loc[monitors['price'].notnull(), 'price'] = monitors['price'].str[1:]
monitors.loc[monitors['price'].notnull(), 'price'] = monitors['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(monitors)):
    if 'picassoRedirect' in monitors['url'][j]:
        ASIN.append(((monitors['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((monitors['url'][j]).split("/")[3])
monitors['ASIN']=ASIN
monitors.loc[monitors['reviews'].notnull(), 'reviews'] = monitors['reviews'].str.replace(',','')
monit_cut = monitors.dropna(how='any',axis=0)
#view DataFrame
monit_cut.loc[monit_cut['reviews'].notnull(), 'reviews'] = monit_cut['reviews'].astype(int)
monit_cut.info()
#view DataFrame
monitors.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6218 entries, 0 to 7187
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       6218 non-null   object
 1   url         6218 non-null   object
 2   rating      6218 non-null   object
 3   reviews     6218 non-null   object
 4   price       6218 non-null   object
 5   search_url  6218 non-null   object
 6   ASIN        6218 non-null   object
dtypes: object(7)
memory usage: 388.6+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,Sceptre IPS 24-Inch Business Computer Monitor ...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.6,1361,119.97,https://www.amazon.com/s?k=monitor&i=computers...,B087QRW81Z
1,ErGear 17”-32” Dual Monitor Desk Mount Stand K...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.5,1378,51.97,https://www.amazon.com/s?k=monitor&i=computers...,B085Y4HW8S
2,SAMSUNG LC27F398FWNXZA SAMSUNG C27F398 27 Inch...,/Samsung-LC27F398FWNXZA-C27F398-Curved-Monitor...,4.7,5114,159.99,https://www.amazon.com/s?k=monitor&i=computers...,B01CX26WIG
3,"Sceptre 20"" 1600x900 75Hz Ultra Thin LED Monit...",/Sceptre-E205W-16003R-Frameless-Speakers-Metal...,4.5,13490,79.97,https://www.amazon.com/s?k=monitor&i=computers...,B07743412C
4,"Sceptre Curved 27"" 75Hz LED Monitor HDMI VGA B...",/Sceptre-Monitor-Speakers-Edge-Less-C275W-1920...,4.7,5667,166.99,https://www.amazon.com/s?k=monitor&i=computers...,B07MTRQ6B3


In [80]:
monit_cut=monit_cut.drop_duplicates(subset='title',inplace=False)
monit_cut[ (monit_cut['price']>50) ].sort_values(by='reviews', ascending=False)['title'].head(50)

5475    NETGEAR - R7000P-100NAS Nighthawk WiFi Router ...
3200    SanDisk 500GB Extreme Portable SSD - Up to 105...
43      Acer SB220Q bi 21.5 Inches Full HD (1920 x 108...
4842    Samsung (MZ-V7E1T0BW) 970 EVO SSD 1TB - M.2 NV...
4648    NETGEAR Cable Modem CM1000 - Compatible with A...
46      HP VH240a 23.8-Inch Full HD 1080p IPS LED Moni...
1184    HDMI to VGA, 10 Pack, Benfei Gold-Plated HDMI ...
5368    NETGEAR Nighthawk Cable Modem Wi-Fi Router Com...
3       Sceptre 20" 1600x900 75Hz Ultra Thin LED Monit...
5632    Western Digital 1TB WD Blue SN550 NVMe Interna...
4227    Raspberry Pi 4 Model B 2019 Quad Core 64 Bit W...
5065    NETGEAR Orbi Whole Home Tri-Band Mesh Wi-Fi 6 ...
41      Acer R240HY bidx 23.8-Inch IPS HDMI DVI VGA (1...
47      HP Pavilion 22cwa 21.5-Inch Full HD 1080p IPS ...
11      BenQ 24 Inch IPS Monitor | 1080P | Proprietary...
7138    Contixo V8-2 7 inch Kids Tablets - Tablet for ...
42      HP 24mh FHD Monitor - Computer Monitor with 23...
6298    Razer 

In [112]:
monit_cut['norm_review']=(monit_cut['rating']*monit_cut['reviews']+6)/(monit_cut['reviews']+2)
monit_cut[ (monit_cut['price']>100) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

363     Portable Monitor - FANGOR 15.6" Full HD 1080P ...
4842    Samsung (MZ-V7E1T0BW) 970 EVO SSD 1TB - M.2 NV...
347     Portable Monitor, Geoyeao 16 Inch EVP-304 Full...
672     MSI 23.6" Full HD 144Hz HDMI DP FreeSync Curve...
5632    Western Digital 1TB WD Blue SN550 NVMe Interna...
6047    SAMSUNG 870 QVO SATA III 2.5" SSD 1TB (MZ-77Q1...
51      Asus VP249QGR 23.8” Gaming Monitor 144Hz Full ...
57      Dell Gaming S2721HGF 27 Inch Curved FHD 144Hz ...
426     MSI Full HD Gaming Non-Glare Super Narrow Beze...
29      Sceptre Curved 27" Gaming Monitor up to 165Hz ...
67      Alienware 25 AW2521HF 24.5 inch Gaming Monitor...
399     USB-C Portable Monitor - 15.6 Inch FHD HDR Fre...
158     Portable Monitor- InnoView 2020 Upgraded 15.6"...
97      Lenovo C27-20 27-Inch WLED Monitor, FHD, IPS, ...
749     Portable Monitor – 2021 Upgraded 15.6 Inch 108...
2492    VEIKK VK1200 Drawing Monitor Full-Laminated Gr...
1555    Raspberry Pi 7" Touch Screen Display with Case...
831     Raspbe

In [146]:
txt='https://www.amazon.com/s?k=mouse&i=computers-intl-ship&ref=nb_sb_noss_1 \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=mouse&i=computers-intl-ship&page='+str(j+2)+'&qid=1613493506&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Mouses.txt", "w")
text_file.write(txt)
text_file.close()

In [147]:
with open('Mouses.txt','r') as urllist, open('search_results_output_4.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=mouse&i=computers-intl-ship&page=106&qid=1613493506&ref=sr_pg_106 must have been blocked by Amazon as the status code was 200


In [6]:
mouses = pd.read_json('search_results_output_4.jsonl', orient='records')
mouses.loc[mouses['rating'].notnull(), 'rating'] = mouses['rating'].str[0:3].astype(float)
mouses.loc[mouses['price'].notnull(), 'price'] = mouses['price'].str[1:]
mouses.loc[mouses['price'].notnull(), 'price'] = mouses['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(mouses)):
    if 'picassoRedirect' in mouses['url'][j]:
        ASIN.append(((mouses['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((mouses['url'][j]).split("/")[3])
mouses['ASIN']=ASIN
mouses.loc[mouses['reviews'].notnull(), 'reviews'] = mouses['reviews'].str.replace(',','')
mice_cut = mouses.dropna(how='any',axis=0)
#view DataFrame
mice_cut.loc[mice_cut['reviews'].notnull(), 'reviews'] = mice_cut['reviews'].astype(int)
mice_cut.info()
#view DataFrame
mouses.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8077 entries, 0 to 8787
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       8077 non-null   object
 1   url         8077 non-null   object
 2   rating      8077 non-null   object
 3   reviews     8077 non-null   object
 4   price       8077 non-null   object
 5   search_url  8077 non-null   object
 6   ASIN        8077 non-null   object
dtypes: object(7)
memory usage: 504.8+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,"WisFox Wireless Computer Mouse, 2.4G Wireless ...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.6,2201,9.99,https://www.amazon.com/s?k=mouse&i=computers-i...,B087LQVH4Y
1,"Bluetooth Mouse, INPHIC Multi-Device Slim Sile...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.2,984,14.59,https://www.amazon.com/s?k=mouse&i=computers-i...,B07ZR9S9X6
2,VicTsing MM057 2.4G Wireless Portable Mobile M...,/VicTsing-Wireless-Portable-Receiver-Adjustabl...,4.6,120106,11.99,https://www.amazon.com/s?k=mouse&i=computers-i...,B013WC0P2A
3,Amazon Basics 3-Button USB Wired Computer Mous...,/AmazonBasics-3-Button-Wired-Computer-1-Pack/d...,4.6,43920,None,https://www.amazon.com/s?k=mouse&i=computers-i...,B005EJH6RW
4,"seenda Wireless Mouse, 2.4G Noiseless Mouse wi...",/seenda-Wireless-Noiseless-Receiver-Portable/d...,4.6,60319,9.99,https://www.amazon.com/s?k=mouse&i=computers-i...,B07H8TJMX7


In [87]:
mice_cut=mice_cut.drop_duplicates(subset='title',inplace=False)
mice_cut[ (mice_cut['price']>5) & mice_cut['title'].str.contains('M5') ].sort_values(by='reviews', ascending=False)['title'].head(50)

59      Logitech M535 Bluetooth Mouse Compact Wireless...
1725    New Logitech Unifying USB Receiver for Mouse M...
2434    3M Wireless Ergonomic Optical Mouse, Patented ...
2312    Perixx PERIPRO-303 1.34 Inches Trackball - Rep...
5593    Corsair M55 RGB PRO, Ambidextrous Multi-Grip O...
993     Acer RF Wireless Optical M501 White Mouse with...
826     Logitech M570 Wireless Trackball Mouse – Ergon...
2781    Logitech M535 Compact Bluetooth Mouse Black 91...
4545    Logitech M510 Wireless Computer Mouse – Comfor...
411     Genuine Dell K251D 6-Button Grey Gray Silver B...
4877               Logitech M570 Wireless Trackball Mouse
4715    DGG ST-M5 12,000 DPI RGB Ambidextrous Wired Ga...
2151    AZIO KM535 - Computer Keyboard and Mouse Combo...
5996    Logitech M570 Wireless Trackball Mouse – Ergon...
7284    M55 Silent Plus 2.4GHz Wireless Mouse, 95% Les...
Name: title, dtype: object

In [110]:
mice_cut['norm_review']=(mice_cut['rating']*mice_cut['reviews']+6)/(mice_cut['reviews']+2)
mice_cut[ (mice_cut['price']>10) ].sort_values(by='norm_review', ascending=False)['title'].head(50)

2033    Ambesonne World Map Mouse Pad, Old Chart Vinta...
2366    Demon Slayer Inosuke Hashibira Anime Mouse Pad...
2512    Demon Slayer Tanjiro Kamado Nezuko Kamado Anim...
842      BT21 Baby Wireless Silent Mouse by Royche (TATA)
1336    Ro-b-lo-x Poster Computer Mouse Pad with Non-S...
2784    SMILESLIM Large Gaming Mouse Pad Extended Pad ...
399             Mickey Mouse 40" Giant Plush Minnie Mouse
603     AUKEY Gaming Mouse Pad Large (31.5 x 11.8 x 0....
1043    Elvira Mistress of The Dark Official Mouse Pad...
2681    Moptmos 3D Mouse Pad Ergonomic Mouse Pad with ...
2349    NJHX Large Gaming Mouse Pad- Waterproof XXL Lo...
5584    BAGSMART Laptop Sleeve Bag Fits 13-13.3 inch N...
3798    Demon Slayer Tanjiro Kamado Anime Mouse Pad La...
927     TEEDOR RGB Gaming Mouse Pad Extended Large LED...
85      ONIKUMA RGB Gaming Mouse Wired, USB Optical Co...
2095    Large Gaming Mouse Pad One Piece Animation 36 ...
3467    Extend JoJo's Bizarre Adventure Mouse Pad 24x1...
1281    Large 

In [152]:
txt='https://www.amazon.com/s?k=keyboard&i=computers-intl-ship&ref=nb_sb_noss_2 \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=keyboard&i=computers-intl-ship&page='+str(j+2)+'&qid=1613493864&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Keyboards.txt", "w")
text_file.write(txt)
text_file.close()



In [150]:
with open('Keyboards.txt','r') as urllist, open('search_results_output_5.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=keyboard&i=computers-intl-ship&page=356&qid=1613493864&ref=sr_pg_356 must have been blocked by Amazon as the status code was 200
Page https://www.amazon.com/s?k=keyboard&i=computers-intl-ship&page=375&qid=1613493864&ref=sr_pg_375 must have been blocked by Amazon as the status code was 200
Page https://www.amazon.com/s?k=keyboard&i=computers-intl-ship&page=398&qid=1613493864&ref=sr_pg_398 must have been blocked by Amazon as the status code was 200


In [7]:
keyboards = pd.read_json('search_results_output_5.jsonl', orient='records')
keyboards.loc[keyboards['rating'].notnull(), 'rating'] = keyboards['rating'].str[0:3].astype(float)
keyboards.loc[keyboards['price'].notnull(), 'price'] = keyboards['price'].str[1:]
keyboards.loc[keyboards['price'].notnull(), 'price'] = keyboards['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(keyboards)):
    if 'picassoRedirect' in keyboards['url'][j]:
        ASIN.append(((keyboards['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((keyboards['url'][j]).split("/")[3])
keyboards['ASIN']=ASIN
keyboards.loc[keyboards['reviews'].notnull(), 'reviews'] = keyboards['reviews'].str.replace(',','')
keyb_cut = keyboards.dropna(how='any',axis=0)
#view DataFrame
keyb_cut.loc[keyb_cut['reviews'].notnull(), 'reviews'] = keyb_cut['reviews'].astype(int)
keyb_cut.info()
#view DataFrame
keyboards.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8571 entries, 0 to 9620
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       8571 non-null   object
 1   url         8571 non-null   object
 2   rating      8571 non-null   object
 3   reviews     8571 non-null   object
 4   price       8571 non-null   object
 5   search_url  8571 non-null   object
 6   ASIN        8571 non-null   object
dtypes: object(7)
memory usage: 535.7+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,"Gaming Keyboard, WisFox USB Wired Keyboard Dur...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.7,69,21.99,https://www.amazon.com/s?k=keyboard&i=computer...,B08GKXKZQY
1,Havit Wired Gaming Keyboard Mouse Combo LED Ra...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.5,12536,25.49,https://www.amazon.com/s?k=keyboard&i=computer...,B016Y2BVKA
2,"Logitech Desktop MK120 Durable, Comfortable, U...",/Logitech-Desktop-Durable-Comfortable-keyboard...,4.6,11056,None,https://www.amazon.com/s?k=keyboard&i=computer...,B003NREDC8
3,NPET K10 Gaming Keyboard USB Wired Floating Ke...,/NPET-Floating-Keyboard-Mechanical-Illuminated...,4.5,11080,21.99,https://www.amazon.com/s?k=keyboard&i=computer...,B01ALLT2W4
4,VicTsing Wireless Keyboard [Enlarged Indicator...,/VicTsing-Wireless-Indicator-Ergonomic-Chromeb...,4.5,884,19.99,https://www.amazon.com/s?k=keyboard&i=computer...,B08C73FX64


In [77]:
keyb_cut=keyb_cut.drop_duplicates(subset='title',inplace=False)
keyb_cut[(keyb_cut['title'].str.contains("Keyboard")) & (keyb_cut['price']>10) ].sort_values(by='reviews', ascending=False)['title'].head(50)

7439    Acer Aspire 5 Slim Laptop, 15.6" Full HD IPS D...
453     Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...
1183    MOSISO MacBook Air 13 inch Case 2020 2019 2018...
2376    TP-Link USB Bluetooth Adapter for PC(UB400), 4...
2831    MOSISO Compatible with MacBook Pro 13 inch Cas...
330     MOSISO Compatible with MacBook Air 13 inch Cas...
52      OMOTON Ultra-Slim Bluetooth Keyboard Compatibl...
1       Havit Wired Gaming Keyboard Mouse Combo LED Ra...
2792    ProCase MacBook Pro 13 Case 2019 2018 2017 201...
1599    MOSISO Case Only Compatible with MacBook Pro R...
84      iPad Keyboard Case for 10.2-inch iPad 8th Gene...
202     iPad Keyboard Case for iPad 2018 (6th Gen) - i...
3       NPET K10 Gaming Keyboard USB Wired Floating Ke...
3104    KECC Laptop Case Compatible with MacBook 12" w...
8       Wireless Keyboard Mouse Combo, Cimetech Compac...
6       Wireless Keyboard Mouse Combo, Cimetech Compac...
19      Razer Huntsman Mini 60% Gaming Keyboard: Faste...
5090    Blade 

In [105]:
keyb_cut['norm_review']=(keyb_cut['rating']*keyb_cut['reviews']+6)/(keyb_cut['reviews']+2)
keyb_cut[ (keyb_cut['price']>10) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

4172    Demon Slayer Tanjiro Kamado Anime Mouse Pad La...
4947    AUKEY Gaming Mouse Pad Large (31.5 x 11.8 x 0....
9498    NJHX Large Gaming Mouse Pad- Waterproof XXL Lo...
4636    TEEDOR RGB Gaming Mouse Pad Extended Large LED...
5346    USB C Hub 4-in-1and USB Type C Adapter,compita...
7974    Pink Flamingo Mouse Pad Customized, Pineapples...
731     Drop + Matt3o MT3 Susuwatari Custom Keycap Set...
425     Retro Typewriter Style Mechanical Keyboard, Wh...
982     Sweet Alice Mechanical Gaming Keyboard, Compac...
8715    King-dom Hearts Desk Mousepad Extra Large Mous...
4705    SOUNDANCE Premium Monitor Stand, Thick Aluminu...
4035    EPOMAKER AKKO NEON Cherry Profile Double-Shot ...
2923    Vaweyaw Demon Slayer Kimetsu No Yaiba Office D...
1355    OMOTON iPad 10.2 8/7th Gen 2020 Keyboard Case,...
996     CP3 Gaming Keyboard RGB Wired Keyboard Anti-gh...
9120    Mammoth High Grade Clear Acrylic Monitor Stand...
8204    Large Mouse Pad, Zpose Mouse Pad Gaming, XXL L...
32      AZIO K

In [154]:
txt='https://www.amazon.com/s?k=dslr+camera&i=electronics-intl-ship&crid=C9K1HHQEVEL8&sprefix=ds%2Ccomputers-intl-ship%2C373&ref=nb_sb_ss_ts-doa-p_3_2 \n'
for j in range(150):
    txt+=('https://www.amazon.com/s?k=dslr+camera&i=electronics-intl-ship&page='+str(j+2)+'&crid=13P2UIK0U581&qid=1613572430&sprefix=dslr+cam%2Caps%2C235&ref=sr_pg_'+str(j+2))
    if j!=149:
        txt+='\n'
text_file = open("Cameras.txt", "w")
text_file.write(txt)
text_file.close()

In [155]:
with open('Cameras.txt','r') as urllist, open('search_results_output_6.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')


In [57]:
cameras = pd.read_json('search_results_output_6.jsonl', orient='records')
cameras.loc[cameras['rating'].notnull(), 'rating'] = cameras['rating'].str[0:3].astype(float)
cameras.loc[cameras['price'].notnull(), 'price'] = cameras['price'].str[1:]
cameras.loc[cameras['price'].notnull(), 'price'] = cameras['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(cameras)):
    if 'picassoRedirect' in cameras['url'][j]:
        ASIN.append(((cameras['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((cameras['url'][j]).split("/")[3])
cameras['ASIN']=ASIN
cameras.loc[cameras['reviews'].notnull(), 'reviews'] = cameras['reviews'].str.replace(',','')
#cameras.loc[cameras['reviews'].notnull(), 'reviews'] = cameras['reviews'].astype(int)
camer_cut = cameras.dropna(how='any',axis=0)
#view DataFrame
camer_cut.loc[camer_cut['reviews'].notnull(), 'reviews'] = camer_cut['reviews'].astype(int)
camer_cut.info()
#view DataFrame
camer_cut

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2846 entries, 0 to 3182
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       2846 non-null   object
 1   url         2846 non-null   object
 2   rating      2846 non-null   object
 3   reviews     2846 non-null   object
 4   price       2846 non-null   object
 5   search_url  2846 non-null   object
 6   ASIN        2846 non-null   object
dtypes: object(7)
memory usage: 177.9+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,Canon EOS Rebel T100 DSLR Camera with 18-55mm ...,/Canon-EOS-Rebel-T100-18-55mm/dp/B087T9B6X5/re...,4.7,206,424.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B087T9B6X5
1,Canon EOS 4000D DSLR Camera with EF-S 18-55mm ...,/Camera-18-55mm-3-5-5-6-Beginners-Bundle/dp/B0...,4.6,46,479.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B0868VLMWK
2,Canon EOS 4000D / Rebel T100 Digital SLR Camer...,/Canon-4000D-Digital-Complete-Accessory/dp/B08...,4.5,32,499.95,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08DQ4QQ6R
3,Panasonic LUMIX FZ300 Long Zoom Digital Camera...,/PANASONIC-Features-Megapixel-2-3-inch-Dustpro...,4.7,1165,397.99,https://www.amazon.com/s?k=dslr+camera&i=elect...,B011PKJI2Y
4,"Panasonic LUMIX FZ80 4K Digital Camera, 18.1 M...",/PANASONIC-Megapixel-20-1200mm-Stabilization-D...,4.5,2622,297.99,https://www.amazon.com/s?k=dslr+camera&i=elect...,B01MS16V42
...,...,...,...,...,...,...,...
3178,"Aoslen 1080P 60fps Audio Video Capture Card, G...",/Aoslen-Camcorder-Streaming-Conference-Broadca...,3.6,33,17.98,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08HGN4FTV
3179,Grizzly Wilderness Medium Modular Gear Bag for...,/Grizzly-Gear-Bag-Waist-Photographers/dp/B00EA...,4.1,6,27.95,https://www.amazon.com/s?k=dslr+camera&i=elect...,B00EA47YFG
3180,Alstar Bahtinov Mask Focusing Mask for Telesco...,/Alstar-Bahtinov-Mask-Focusing-Telescopes/dp/B...,5.0,1,22.99,https://www.amazon.com/s?k=dslr+camera&i=elect...,B077FMLDMX
3181,Canon PowerShot ELPH 360 HS Digital Camera Bal...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,3.9,6,249.95,https://www.amazon.com/s?k=dslr+camera&i=elect...,B07ZBMJ417


In [58]:
camer_cut=camer_cut.drop_duplicates(subset='title',inplace=False)
camer_cut[(camer_cut['title'].str.contains("DSLR")) & (camer_cut['price']>100) ].sort_values(by='reviews', ascending=False)

,title,url,rating,reviews,price,search_url,ASIN
287,Sigma 17-50mm f/2.8 EX DC OS HSM FLD Large Ape...,/Sigma-17-50mm-Aperture-Standard-Digital/dp/B0...,4.7,3987,329.99,https://www.amazon.com/s?k=dslr+camera&i=elect...,B003A6H27K
28,DJI Ronin-S - Camera Stabilizer 3-Axis Gimbal ...,/DJI-Ronin-S-Handheld-Stabilizer-Mirrorless/dp...,4.6,2427,549.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B07D519QXH
55,Nikon D3400 DSLR Camera with AF-P DX NIKKOR 18...,/Nikon-18-55mm-3-5-5-6G-70-300mm-4-5-6-3G/dp/B...,4.7,2320,784.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B01KITZRBE
344,Nikon AF S NIKKOR 85mm f/1.8G Fixed Lens with ...,/Nikon-NIKKOR-Fixed-Focus-Cameras/dp/B006TAP09...,4.8,1687,476.95,https://www.amazon.com/s?k=dslr+camera&i=elect...,B006TAP096
2404,Tamron Auto Focus 18-270mm f/3.5-6.3 VC PZD Al...,/Tamron-18-270mm-3-5-6-3-Cameras-B008N/dp/B004...,4.3,1548,399.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B004FN1W2I
...,...,...,...,...,...,...,...
17,Canon EOS 2000D / Rebel T7 Digital SLR Camera ...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,5.0,1,549.95,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08S5FD42N
1002,Think Tank Photo Backstory 13 Camera Backpack ...,/Think-Tank-Backstory-Backpack-Mirrorless/dp/B...,5.0,1,189.75,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08JG6G1SY
2349,Canon EOS 2000D / Rebel T7 Digital DSLR Camera...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,5.0,1,519.99,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08MVBJGKX
1202,Saramonic Blink500 Pro B2W Mini Dual-Channel S...,/Saramonic-Dual-Channel-Microphone-Mirrorless-...,5.0,1,299.00,https://www.amazon.com/s?k=dslr+camera&i=elect...,B08Q7H7PP7


In [104]:
camer_cut['norm_review']=(camer_cut['rating']*camer_cut['reviews']+6)/(camer_cut['reviews']+2)
camer_cut[ (camer_cut['price']>100) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

1610    Canon EF 100mm f/2.8L IS USM Macro Lens for Ca...
1064    Canon EF 70-200mm f/2.8L IS III USM Lens for C...
2759    Canon EOS R5 Full-Frame Mirrorless Camera with...
2728                         Canon EF 50mm f/1.8 STM Lens
2594                       Canon EF-S 24mm f/2.8 STM Lens
1338    Canon EF-S 55-250mm f/4.0-5.6 IS II Telephoto ...
191     Canon EOS M50 Mirrorless Vlogging Camera Kit w...
2685    Canon EF 85mm f/1.8 USM Medium Telephoto Lens ...
344     Nikon AF S NIKKOR 85mm f/1.8G Fixed Lens with ...
641     Sony a7 III (ILCE7M3K/BQ) Full-frame Mirrorles...
355     Neewer 2-in-1 Convertible Wheeled Camera Backp...
576     Sony Alpha a6400 Mirrorless Camera: Compact AP...
1206    Canon EF-S 55-250mm F4-5.6 is STM Lens for Can...
12      Canon DSLR Camera [EOS 90D] with Built-in Wi-F...
964     Fujifilm X-T30 Mirrorless Digital Camera w/XF1...
2749    Canon Battery Grip for EOS 70D Digital SLR Camera
50      Nikon D5600 DSLR Camera with 18-55mm VR Lens +...
1707    Cayer 

In [158]:
txt='https://www.amazon.com/s?k=smartphone&i=electronics-intl-ship&ref=nb_sb_noss \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=smartphone&i=electronics-intl-ship&page='+str(j+2)+'&qid=1613494283&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Smartphones.txt", "w")
text_file.write(txt)
text_file.close()

In [159]:
with open('Smartphones.txt','r') as urllist, open('search_results_output_7.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=smartphone&i=electronics-intl-ship&page=390&qid=1613494283&ref=sr_pg_390 must have been blocked by Amazon as the status code was 200


In [9]:
smartphones = pd.read_json('search_results_output_7.jsonl', orient='records')
smartphones.loc[smartphones['rating'].notnull(), 'rating'] = smartphones['rating'].str[0:3].astype(float)
smartphones.loc[smartphones['price'].notnull(), 'price'] = smartphones['price'].str[1:]
smartphones.loc[smartphones['price'].notnull(), 'price'] = smartphones['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(smartphones)):
    if 'picassoRedirect' in smartphones['url'][j]:
        ASIN.append(((smartphones['url'][j]).split("%")[3])[2:])
    else:
        ASIN.append((smartphones['url'][j]).split("/")[3])
smartphones['ASIN']=ASIN
smartphones.loc[smartphones['reviews'].notnull(), 'reviews'] = smartphones['reviews'].str.replace(',','')
smartph_cut = smartphones.dropna(how='any',axis=0)
#view DataFrame
smartph_cut.loc[smartph_cut['reviews'].notnull(), 'reviews'] = smartph_cut['reviews'].astype(int)
smartph_cut.info()
#view DataFrame
smartphones.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6695 entries, 0 to 7199
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       6695 non-null   object
 1   url         6695 non-null   object
 2   rating      6695 non-null   object
 3   reviews     6695 non-null   object
 4   price       6695 non-null   object
 5   search_url  6695 non-null   object
 6   ASIN        6695 non-null   object
dtypes: object(7)
memory usage: 418.4+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,"TCL 10L, Unlocked Android Smartphone with 6.53...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.4,1462,249.99,https://www.amazon.com/s?k=smartphone&i=electr...,B087LYQ22N
1,"Unlocked Cell Phones, Blackview BV5900 Rugged ...",/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.0,500,149.99,https://www.amazon.com/s?k=smartphone&i=electr...,B08GR8MYJJ
2,Samsung Electronics Samsung Galaxy S21 5G | Fa...,/Samsung-Electronics-Smartphone-Pro-Grade-SM-G...,4.6,6,799.99,https://www.amazon.com/s?k=smartphone&i=electr...,B08N2FRMPN
3,Samsung Galaxy S20 FE 5G | Factory Unlocked An...,/Samsung-Factory-Unlocked-Smartphone-Pro-Grade...,4.6,2982,599.99,https://www.amazon.com/s?k=smartphone&i=electr...,B08FYTSXGQ
4,"OUKITEL WP8 Pro (2020) Rugged Smartphone, 6.49...",/OUKITEL-WP8-Pro-Smartphone-Fingerprint/dp/B08...,4.1,424,189.99,https://www.amazon.com/s?k=smartphone&i=electr...,B08HRJM99F


In [96]:
smartph_cut=smartph_cut.drop_duplicates(subset='title',inplace=False)
smartph_cut[ (smartph_cut['title'].str.contains("Apple")) ].sort_values(by='reviews', ascending=False)['title'].head(50)

4711    Apple iPhone 8, 64GB, Gold - Fully Unlocked (R...
1092    SHANSHUI Cell Phone Wallet, ID Credit Card Hol...
1173    Macally Car Headrest Mount Holder for Apple iP...
1137    AEDILYS 12 Pack Universal Capacitive Stylus Pe...
2111    Cup Mount Holder iKross 2-in-1 Tablet and Smar...
920     Fosmon Mini Bluetooth Keyboard (QWERTY Keypad)...
1868    DIGIANT 50 Inch Aluminum Camera Phone Tripod+ ...
1382    Bone Run Tie Running Armband Phone Holder for ...
615     Philips Universal Remote Control with Smartpho...
3566    CableCreation 6 Feet 3.5mm Auxiliary Audio Cab...
2307    BENTOBEN Compatible with Apple Watch Stand Air...
6774    Sim Card Tray Pin Eject Removal Tool Needle Op...
3894    Wireless Charger,Kertxin 4 in 1 Charging Stati...
5374    Apple MFi Certified Lightning Cable (2-Pack), ...
4029    Aux Cable, CableCreation [2-Pack 1.5 FT] Flat ...
4639    Fosmon Youth Series Kids Stylus Pen for Apple ...
1982    Stylus Pen, ARYKX 10 Pack of Fiber Mesh Tip St...
6728    kwmobi

In [100]:
smartph_cut['norm_review']=(smartph_cut['rating']*smartph_cut['reviews']+6)/(smartph_cut['reviews']+2)
smartph_cut[ (smartph_cut['price']>100) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

65      Xiaomi Redmi Note 9 4GB RAM + 128GB, 48MP Quad...
5728    DJI Mini 2 – Ultralight and Foldable Drone Qua...
534     WiFi Projector, Full HD Native 1080P Projector...
3791    BONAOK 2021 Karaoke Microphone, Portable Wirel...
2264    Fujifilm Instax Share SP-3 Smartphone Printer ...
4315    2.4GHz Wireless Microphone System, Saramonic B...
6468    Canon Selphy CP1300 Wireless Compact Photo Pri...
4947    Fujitsu ScanSnap iX1500 Color Duplex Document ...
2856    HP Sprocket Photo Printer (2nd Edition) Instan...
6879    Comica CVM-VM20 Camera Shotgun Microphone with...
75      GRC Mini Projector, Portable WiFi Movie Projec...
5106    JEEMAK Mini Video Projector,Outdoor Movie Proj...
588     YC Onion Teleprompter for Tablet Smartphone Ca...
2756    PENTAX Monocular VM 6x21 WP - Complete Kit. In...
4002    Wireless Lavalier Microphone System, COMICA Bo...
5425    Gosky HD Spotting Scope 20-60x 80mm with Tripo...
4433    Fujitsu ScanSnap iX1600 Versatile Cloud Enable...
4052    Telepr

In [161]:
txt='https://www.amazon.com/s?k=headphones&i=electronics-intl-ship&ref=nb_sb_noss_2 \n'
for j in range(400):
    txt+=('https://www.amazon.com/s?k=headphones&i=electronics-intl-ship&page='+str(j+2)+'&qid=1613494380&ref=sr_pg_'+str(j+2))
    if j!=399:
        txt+='\n'
text_file = open("Headphones.txt", "w")
text_file.write(txt)
text_file.close()

In [162]:
with open('Headphones.txt','r') as urllist, open('search_results_output_8.jsonl','w') as outfile:
    outfile.write('[')
    for url in urllist.read().splitlines():
        data = scrape(url) 
        if data:
            #print(data)
        #sleep(4)
            
            for product in data['products']:
                product['search_url'] = url
                #print("Saving Product: %s"%product['title'])
                json.dump(product,outfile)
                outfile.write(", \n")
                #sleep(2)
    outfile.write(']')

Page https://www.amazon.com/s?k=headphones&i=electronics-intl-ship&page=25&qid=1613494380&ref=sr_pg_25 must have been blocked by Amazon as the status code was 200


In [10]:
headphones = pd.read_json('search_results_output_8.jsonl', orient='records')
headphones.loc[headphones['rating'].notnull(), 'rating'] = headphones['rating'].str[0:3].astype(float)
headphones.loc[headphones['price'].notnull(), 'price'] = headphones['price'].str[1:]
headphones.loc[headphones['price'].notnull(), 'price'] = headphones['price'].str.replace(',','').astype(float)
ASIN=[]
for j in range(len(headphones)):
    #print(j)
    if headphones['url'][j]:
        if 'picassoRedirect' in headphones['url'][j]:
            ASIN.append(((headphones['url'][j]).split("%")[3])[2:])
        else:
            ASIN.append((headphones['url'][j]).split("/")[3])
    else:
        ASIN.append(0)
headphones['ASIN']=ASIN
headphones.loc[headphones['reviews'].notnull(), 'reviews'] = headphones['reviews'].str.replace(',','')
headph_cut = headphones.dropna(how='any',axis=0)
#view DataFrame
headph_cut.loc[headph_cut['reviews'].notnull(), 'reviews'] = headph_cut['reviews'].astype(int)
headph_cut.info()
headph_cut.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7845 entries, 0 to 8761
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       7845 non-null   object
 1   url         7845 non-null   object
 2   rating      7845 non-null   object
 3   reviews     7845 non-null   object
 4   price       7845 non-null   object
 5   search_url  7845 non-null   object
 6   ASIN        7845 non-null   object
dtypes: object(7)
memory usage: 490.3+ KB


,title,url,rating,reviews,price,search_url,ASIN
0,Artix CL750 Foldable Noise Isolating On Ear He...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.4,3532,24.99,https://www.amazon.com/s?k=headphones&i=electr...,B087JVV8FK
1,COWIN E7 Active Noise Cancelling Headphones Bl...,/gp/slredirect/picassoRedirect.html/ref=pa_sp_...,4.4,66158,59.99,https://www.amazon.com/s?k=headphones&i=electr...,B019U00D7K
3,Anker Soundcore Life Q20 Hybrid Active Noise C...,/Soundcore-Cancelling-Headphones-Wireless-Blue...,4.5,21152,49.99,https://www.amazon.com/s?k=headphones&i=electr...,B07NM3RSRQ
4,Mpow CH6S Kids Headphones with Microphone Over...,/Mpow-Headphones-Over-Ear-Function-Cellphone/d...,4.7,25420,18.59,https://www.amazon.com/s?k=headphones&i=electr...,B07PD9V78L
5,"Bluetooth Over-Ear Headphones, Zihnic Foldable...",/Bluetooth-Over-Ear-Headphones-Foldable-Prolon...,4.4,31825,25.99,https://www.amazon.com/s?k=headphones&i=electr...,B07K5214NZ


In [78]:
headph_cut=headph_cut.drop_duplicates(subset='title',inplace=False)
headph_cut[ (headph_cut['price']>10) ].sort_values(by='reviews', ascending=False)['title'].head(50)

11      TOZO T10 Bluetooth 5.0 Wireless Earbuds with W...
167     PANASONIC ErgoFit Earbud Headphones with Micro...
43      TOZO T6 True Wireless Earbuds Bluetooth Headph...
7       TOZO T6 True Wireless Earbuds Bluetooth Headph...
1304    RUNMUS Gaming Headset Xbox One Headset with 7....
1       COWIN E7 Active Noise Cancelling Headphones Bl...
9       LETSCOM Bluetooth Headphones V5.0 IPX7 Waterpr...
40      Mpow Flame Bluetooth Headphones V5.0 IPX7 Wate...
8243    Bluetooth Headphones, True Wireless Earbuds St...
3526    Samsung Galaxy Buds Plus, True Wireless Earbud...
6680    INIU Portable Charger, USB C Slimmest & Lighte...
2694    Turtle Beach Recon 70 White Gaming Headset for...
1349    Turtle Beach Recon 70 Gaming Headset for Ninte...
6499    Mpow Car Phone Mount, Dashboard Car Phone Hold...
337     Wireless Earbuds, Mpow M30 in-Ear Bluetooth He...
6064                           OnePlus Bullets Wireless Z
5981    Tablet Stand Adjustable, Lamicall Tablet Stand...
408     2019 U

In [102]:
headph_cut['norm_review']=(headph_cut['rating']*headph_cut['reviews']+6)/(headph_cut['reviews']+2)
headph_cut[ (headph_cut['price']>10) ].sort_values(by='norm_review', ascending=False)['title'].head(30)

6160    True Wireless Earbuds Bluetooth 5.0 Headphones...
168     True Wireless Earbuds Bluetooth Headphones, TW...
214     Lasuney Active Noise Cancelling Headphones APT...
184     Losei Bluetooth Headphones, True Wireless Earb...
1960    BC Master Headphone Stand Hanger Foldable Alum...
7432    Willnorn 12-Outlets Power Strip Surge Protecto...
2926    Losei D65 Bluetooth Headphones Wireless for Sp...
905     PeohZarr Gaming Headset Xbox One Headset PS4 H...
346     Headset Holder Under Desk Headphone Hook Folda...
8665    Rick and Morty Air Pods 2 and Air pod Pro Case...
7472    Galaxy Buds+ Plus BTS Edition | True Wireless ...
5981    Tablet Stand Adjustable, Lamicall Tablet Stand...
635     Bluetooth Headphones 5.0,Smart Touch Wireless ...
74      Sony MDR7506 Professional Large Diaphragm Head...
6072    2 PACK 6.6FT+6.6FT AINOPE USB 3.0 Extension Ca...
5172    10FT USB 3.0 Extension Cable Type A Male to Fe...
6383    Blade Hawks RGB Gaming Mouse Pad, LED Soft Ext...
7112    9 Pack